In [2]:
import random 
import numpy as np
import pygame
# from random import choice,random
import numpy as np
import matplotlib.pyplot as plt 
from PIL import Image
import cv2


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
class Simulator:
    def __init__(self,stickiness,particles):
        print("one")
        self.width,self.height = 1000,1000   # the window size of simulation 
        self.startX,self.startY = round(self.width/2),round(self.height/2) # The centre point where the particles will start to aggregate
        self.x_coordinate,self.y_coordinate = self.startX, self.startY
        
        # these are created to optimise the model. Particles introduced at the corners were taking a lot of time to reach the centre and thus to increase the probability of particles finding the centre point the window size is reduced.


        self.minX,self.minY = self.x_coordinate,self.y_coordinate
        self.maxX, self.maxY = self.x_coordinate,self.y_coordinate

         
        self.buffer = 20

        # particle number is the total number of particles in the simulation. In our case it is 80,000 
        self.particleNumber = 0
        

        # these define the boundaries of the window. It keeps on increasing as the contour/simulation increases in size
        self.domainMinX = self.startX - self.buffer
        self.domainMaxX = self.startX + self.buffer
        self.domainMinY = self.startY - self.buffer
        self.domainMaxY = self.startY + self.buffer
        # defines the stickiness coefficient 
        self.stickness = stickiness

        # would work as an iterator to keep the number of particles in the simulation at a specific moment
        self.particles = particles
        # to update the value of a cell from 0 to 1 when a particle sticks to the contour
        self.update = False
        # the working array 
        self.image_array = np.zeros((self.width,self.height))
        # to keep the simulation going 
        self.isRunning = True
        # to start of the middle with a point 
        self.image_array[self.startX,self.startY] = 1

    def particle_movement(self):
        # number of choices the particles has during its random walk - North, South, East, West and NS,SE,NW,SW
        newDir = random.choice(((0,1),(0,-1),(1,0),(-1,0),(1,1),(1,-1),(-1,1),(-1,-1)))

        dX,dY = newDir

        # the new position of particle after random walk
        newX = self.x_coordinate + dX
        newY = self.y_coordinate + dY

        # this is to keep the particles within the bundary. If it crosses from one part of the boundary we introduce the particles from opposite side of the boundary.
        if newX < self.domainMinX:
            newX = self.domainMaxX

            
        if newX > self.domainMaxX :
            newX = self.domainMinX
        
        if newY < self.domainMinY:
            newY = self.domainMaxY
            
        if newY > self.domainMaxY :
            newY = self.domainMinY 


        # this checks if the moving particle has encountered another particle and uses probability of stickiness to decide whether it wants to stick
            
        if self.image_array[newX,newY] == 1 and  random.random() < (self.stickness*0.001):   
            # update if true and convert the cell from 0 to 1 
            self.update = True
            # add another particle to number of particle in the contour
            self.particleNumber += 1 

            # Saving the .npy and .png file at each 1000 particle 
            if self.particleNumber%1000 == 0:
                outpath = '/Users/91706/Locus'
                save_fname1 = os.path.join(outpath, os.path.basename("stickness"+ str(self.stickness)+ "particles" + str(self.particleNumber))+'.npy')
                np.save(save_fname1,self.image_array)
                save_fname2 = os.path.join(outpath, os.path.basename("stickness"+ str(self.stickness)+ "particles" + str(self.particleNumber))+'.png')
                plt.imshow(self.image_array)
                plt.savefig(save_fname2)
            
            # stopping the simulation if number of particle is 80,000
            if self.particleNumber == self.particles*10000:
                self.isRunning = False
            
            # This is to increase the boundary of the domain from which particles are introduced 

            if self.x_coordinate < self.minX:
                self.minX = self.x_coordinate

            if self.x_coordinate > self.maxX:
                self.maxX = self.x_coordinate

            if self.y_coordinate < self.minY:
                self.minY = self.y_coordinate

            if self.y_coordinate > self.maxY:
                self.maxY = self.y_coordinate

            # making sure that the value doesnt shoot the total size of the array
            self.domainMinX = max([self.minX-self.buffer,1])
            self.domainMaxX = min([self.maxX + self.buffer, self.width-1])        
            self.domainMinY = max([self.minY-self.buffer,1])
            self.domainMaxY = min([self.maxY + self.buffer, self.height-1])

           
        else:
            # keep the particle moving on 
            self.update = False
            self.x_coordinate,self.y_coordinate = newX,newY
            
            
    def update_array(self):

        # update the cell to one if the particle has stuck
        if self.update:            
            self.image_array[self.x_coordinate,self.y_coordinate] = 1


            self.update = False
            # randomly decide which side/boundary from which the particle will be introduced 
            boundary_particle = random.choice((1,2,3,4))    

            if (boundary_particle == 1):
                self.x_coordinate = self.domainMinX
                # introducing y from any point on the y surface keeping x = domainMinX
                self.y_coordinate = int(random.uniform(self.domainMinY,self.domainMaxY))

            elif boundary_particle == 2 :
                self.x_coordinate = int(random.uniform(self.domainMinX,self.domainMaxX))
                self.y_coordinate = self.domainMaxY 

            elif boundary_particle == 3 :
                self.x_coordinate = self.domainMaxX
                self.y_coordinate = int(random.uniform(self.domainMinY,self.domainMaxY))

            elif boundary_particle == 4 :
                self.x_coordinate = int(random.uniform(self.domainMinX,self.domainMaxX))
                self.y_coordinate = self.domainMinY 


    def run(self):            
        while self.isRunning:
            self.particle_movement()
            self.update_array()
        # returing the final image array 

        return self.image_array


        
        
import os 
# dict = {}
for stickiness in range(1,51,2):
    new_simulator = Simulator(stickiness,8)
    ki = new_simulator.run()
        